In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import ezhc as hc

from IPython.display import display, HTML
from notebook_wide_screen import WideScreen
from ezaggrid import AgGrid

from sgmarkets_api_auth import Api

In [ ]:
WideScreen(width='95%', verbose=False).add_css()

<h3 markdown="2" style="margin-left: 0%; color:#636363;">Data Source</h3>

If the authentication fails follow the [instructions](https://gitlab.com/sgmarkets/sgmarkets-api-auth#3-user-guide) to set up your credentials.

In [ ]:
a = Api(verbose=False, update_token=True)

In [ ]:
url = 'https://analytics-api.sgmarkets.com/syntheticfinancingtrading/v1/availabilities'
start_date = dt.datetime.today() - pd.tseries.offsets.BDay(0)
end_check_date = start_date - dt.timedelta(days=14)
res = {'availabilities': []}

while not len(res['availabilities']) > 0:
    formated_date = start_date.strftime('%Y-%m-%d')
    params = {'startDate': formated_date, 'pageSize': '100000'}
    res = a.get(url, payload=params)
    if start_date < end_check_date:
        break
    else:
        start_date = start_date - pd.tseries.offsets.BDay(1)
df = pd.DataFrame(res['availabilities'])
df = df[[c for c in df.columns if not c.startswith('Unnamed')]]
df = df.dropna(axis=0, how='all')

In [ ]:
def get_df_bubble(df_input, nominal_col, country_col):
    # Keep only the required colunms
    df = df_input.copy()
    df = df[[country_col, nominal_col]]
    df[nominal_col] = df.apply(lambda x: round(x[nominal_col], 0), axis=1)
    
    # Create agregate functions
    df = df.groupby([country_col])[nominal_col].agg(['sum','count'])
    df['Average Nominal'] = round(df['sum'] / df['count'],0)
    df = df.rename(columns={'count': 'Nb Stocks', 'sum': 'Nominal'})
    df = df.set_index(['Nb Stocks', 'Nominal'], append=True)
    
    return df

def get_df_nature(df_input):
    df = df_input.copy()
    df['nature'] = np.where(df['hardToBorrow'], 'HTB', 'GC')
    df = df[['market', 'nature']]
    df = df.groupby(['market', 'nature']).size().reset_index(name="Nb Stocks")
    df = df.pivot(index='market', columns='nature', values='Nb Stocks')
    return df

def get_df_table(df_input):
    df = df_input.copy()
    df['index'] = df.index.values
    df['sector'] = df.apply(lambda x: str(x['sector']).replace(',',';'), axis=1)
    return df

def build_centered_html(g):
    html = """<div align="center">{}</div>""".format(g.html())
    return html

def plot_series_bubble_html(df_bubble):
    g = hc.Highcharts()
    g.exporting = False
    g.chart.type = 'bubble'
    g.chart.width = 750
    g.chart.height = 500
    g.chart.spacingLeft = 150*0  
    
    g.chart.zoomType = 'xy'
    g.plotOptions.bubble.minSize = 20
    g.plotOptions.bubble.maxSize = 60
    g.legend.enabled = True
    g.title.text = ''
    g.xAxis.title.text = 'Number of stock'
    g.yAxis.title.text = 'Nominal (EUR)'
    g.yAxis.type= 'logarithmic'

    g.series = hc.build.series_bubble(df_bubble)

    html = build_centered_html(g)
    return html

def plot_series_bar_html(df_nature):
    g = hc.Highcharts()
    g.exporting = False
    g.chart.type = 'bar'
    g.chart.width = 750
    g.chart.height = 500
    g.chart.spacingLeft = 150
    # g.chart.spacingRight = 300
    g.title.text = ''
    g.xAxis.title.text = 'Country'
    g.yAxis.title.text = 'Number of stock'
    g.plotOptions.bar.stacking = 'normal'
    g.xAxis.categories = list(df_nature.index)
    g.series = hc.build.series(df_nature)

    html = build_centered_html(g)
    return html

def build_aggrid_table(df_table):
    cols = df_table.columns.values
    cols = list(cols)
    list_first_order = ['index', 'bbg', 'quantity', 'nominalEur']
    for elt in list_first_order:
        cols.insert(list_first_order.index(elt), cols.pop(cols.index(elt)))
    df_table = df_table.reindex(columns=cols)

    list_nominal = ['index', 'quantity', 'nominalCcy', 'nominalEur', 'nominalGbp', 'nominalUsd']
    columnDefs = []
    for elt in df_table.columns:
        if elt in list_nominal:
            columnDefs.append({'headerName': elt, 'field': elt, 'width': 150, 
                               'valueFormatter': 'Math.round(value).toLocaleString("en")'})
        else:
            columnDefs.append({'headerName': elt, 'field': elt, 'width': 150})

    cellStyle = {'text-align': 'center'}

    grid_options = {
        'columnDefs': columnDefs,
        'enableFilter': True,
        'enableSorting': True,
        'enableColResize': True,
        'rowStyle' : cellStyle,
        'pagination': True,
        'paginationPageSize' : 20
    }

    ag = AgGrid(grid_data=df_table,
                grid_options=grid_options,
                css_rules=None,
                width=990,
                height=560,
                index=False,
                quick_filter=True,
                export_csv=True,
                export_excel=False,
                theme='ag-theme-fresh')
    return ag

<h2 markdown="2" style="margin-left: 5%; color:#636363;">Equity Availability</h2>

<h3 markdown="3" style="margin-left: 5%; color:#636363;">1 - Country Availability</h3>

In [ ]:
df_country = get_df_bubble(df,'nominalEur', 'market')
html_countty = plot_series_bubble_html(df_country)
HTML(html_countty)

<h3 markdown="3" style="margin-left: 5%; color:#636363;">2 - Sector Availability</h3>

In [ ]:
df_sector = get_df_bubble(df,'nominalEur', 'sector')
html_sector = plot_series_bubble_html(df_sector)
HTML(html_sector)

<h3 markdown="3" style="margin-left: 5%; color:#636363;">3 - GC/HTB availibility</h3>

In [ ]:
df_nature = get_df_nature(df)
html_nature = plot_series_bar_html(df_nature)
HTML(html_nature)

<h3 markdown="3" style="margin-left: 5%; color:#636363;">4 - Data Summary</h3>

In [ ]:
df_table = get_df_table(df)
ag = build_aggrid_table(df_table)
ag.show()